In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

In [2]:
apple = pd.read_csv("AAPL.csv")
amazon = pd.read_csv("AMZN.csv")
google = pd.read_csv("GOOG.csv")
google_l = pd.read_csv("GOOGL.csv")
microsoft = pd.read_csv("MSFT.csv")
tesla = pd.read_csv("TSLA.csv")

In [3]:
def paint(data):
    data = data.iloc[::-1]
    data = data.reset_index(drop=True)
    data['Label'] = np.where(data['adj close'].shift(-1) > data['adj close'], 1, 0)
    #data['Label'] = (data['close'] > data['open']).astype(int)
    return data
apple = paint(apple)
amazon = paint(amazon)
google = paint(google)
google_l = paint(google_l)
microsoft = paint(microsoft)
tesla = paint(tesla)

# print(apple)

In [4]:
tweets = pd.read_csv("D:\johnny的文件夹\FDU 课程\大二下\大数据商务分析\Final project\data\Tweet\Tweet.csv")
company_tweets = pd.read_csv(
    "D:\johnny的文件夹\FDU 课程\大二下\大数据商务分析\Final project\data\Tweet\Company_Tweet.csv")
tweets.drop('writer', axis=1, inplace=True)
df_tweets = pd.merge(tweets, company_tweets, on='tweet_id', how='left')
df_tweets['post_date'] = pd.to_datetime(df_tweets['post_date'], unit='s')
df_tweets['post_date'] = df_tweets['post_date'].dt.strftime('%Y-%m-%d')
df_tweets.drop('tweet_id', axis=1, inplace=True)


In [8]:
def explore(company,df):
    df1 = df[df['ticker_symbol'] == company]
    comment = df1.groupby('post_date')['comment_num'].sum()
    like = df1.groupby('post_date')['like_num'].sum()
    retweet = df1.groupby('post_date')['retweet_num'].sum()
    df1 = pd.concat([comment, like, retweet], axis=1)
    df1 = df1.reset_index()
    return df1
apple_tweets = explore('AAPL',df_tweets)
amazon_tweets = explore('AMZN',df_tweets)
google_tweets = explore('GOOG',df_tweets)
googlel_tweets = explore('GOOGL',df_tweets)
microsoft_tweets = explore('MSFT',df_tweets)
tesla_tweets = explore('TSLA',df_tweets)

# print(apple_tweets)

In [9]:
def make_stock(stock, tweet):
    stock['date'] = pd.to_datetime(stock['date'])
    tweet['post_date'] = pd.to_datetime(tweet['post_date'])

    start_date = '2015-01-01'
    end_date = '2019-12-31'
    stock = stock.loc[(stock['date'] >= start_date) & (stock['date'] <= end_date)]
    tweet.rename(columns={'post_date': 'date'}, inplace=True)
    result = pd.merge(stock, tweet, on='date', how='left')
    return result
#结合了stock price和tweets的数据在此处！
apple_stock = make_stock(apple, apple_tweets).dropna()
amazon_stock = make_stock(amazon, amazon_tweets).dropna()
google_stock = make_stock(google, google_tweets).dropna()
googlel_stock = make_stock(google_l, googlel_tweets).dropna()
microsoft_stock = make_stock(microsoft, microsoft_tweets).dropna()
tesla_stock = make_stock(tesla, tesla_tweets).dropna()


In [10]:
apple_stock

,date,open,high,low,close,adj close,volume,Label,comment_num,like_num,retweet_num
0,2015-01-02,111.389999,111.440002,107.349998,109.330002,100.216454,53204600,0,102.0,437.0,1834.0
1,2015-01-05,108.290001,108.650002,105.410004,106.250000,97.393181,64285500,1,145.0,593.0,413.0
2,2015-01-06,106.540001,107.430000,104.629997,106.260002,97.402374,65797100,1,120.0,468.0,227.0
3,2015-01-07,107.199997,108.199997,106.699997,107.750000,98.768150,40105900,1,102.0,567.0,210.0
4,2015-01-08,109.230003,112.150002,108.699997,111.889999,102.563072,59364500,1,272.0,857.0,498.0
...,...,...,...,...,...,...,...,...,...,...,...
1253,2019-12-24,284.690002,284.890015,282.920013,284.269989,283.596924,12119700,1,142.0,998.0,208.0
1254,2019-12-26,284.820007,289.980011,284.700012,289.910004,289.223602,23280300,0,258.0,1945.0,410.0
1255,2019-12-27,291.119995,293.970001,288.119995,289.799988,289.113831,36566500,1,152.0,892.0,150.0
1256,2019-12-30,289.459991,292.690002,285.220001,291.519989,290.829773,36028600,1,334.0,2149.0,773.0


In [21]:
from sklearn.preprocessing import MinMaxScaler
closing = apple_stock['adj close'].values.reshape(-1,1)
scaler = MinMaxScaler()
scaled_prices = scaler.fit_transform(closing)

print(scaled_prices)

[[0.07231139]
 [0.0587224 ]
 [0.05876665]
 ...
 [0.98151305]
 [0.98977223]
 [1.        ]]


In [20]:
train_size = int(len(scaled_prices) * 0.8)
train_prices = scaled_prices[:train_size]
test_prices = scaled_prices[train_size:]

print(train_prices)

[[0.07231139]
 [0.0587224 ]
 [0.05876665]
 ...
 [0.32903098]
 [0.3361744 ]
 [0.33702584]]


In [38]:
def create(data,step=1):
    X,y = [],[]
    for i in range(len(data)-step):
        X.append(data[i:(i+step),0])
        y.append([int(data[i + step + j + 1] > data[i + step + j] if i + step + j + 1 < len(data) else 0) for j in range(7)])
    return np.array(X),np.array(y)
X_train, y_train = create(train_prices,10)
X_test, y_test = create(test_prices,10)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print(y_train.shape)

(994, 7)


C:\Users\johnn\AppData\Local\Temp\ipykernel_856\939061209.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y.append([int(data[i + step + j + 1] > data[i + step + j] if i + step + j + 1 < len(data) else 0) for j in range(7)])


In [45]:
import tensorflow as tf
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
# model.add(Dense(units = 128))
model.add(Dense(units=7))
model.compile(loss='mse', optimizer='adam')
model.compile(optimizer='adam', loss='mse',
              metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
model.fit(X_train, y_train, epochs=10, batch_size=128)

Epoch 1/10


C:\Users\johnn\PycharmProjects\Deep Learning\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.4854 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.4037 - precision_7: 0.2007 - recall_7: 0.0038
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.2913 - precision_7: 0.5020 - recall_7: 0.3269
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.2758 - precision_7: 0.5059 - recall_7: 0.4422
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.2688 - precision_7: 0.4967 - recall_7: 0.2898
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.2591 - precision_7: 0.5172 - recall_7: 0.4601
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.2562 - precision_7: 0.5054 - recall_7: 0.4789
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.2530 - precision_7: 0.5158 - recall_7: 0.4523
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.2520 - precision_7: 0.5097 - recall_7: 0.5523
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 

In [46]:
loss,precision,recall = model.evaluate(X_test, y_test)
f1 = 2 * (precision * recall) / (precision + recall)
print(precision, recall, f1)

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.2454 - precision_7: 0.5895 - recall_7: 1.0000
0.5696502923965454 1.0 0.7258308365321324


## 包括更多的LSTM

In [67]:
closing = apple_stock['adj close'].values.reshape(-1,1)
like = apple_stock['like_num'].values.reshape(-1,1)
comment = apple_stock['comment_num'].values.reshape(-1,1)
retweet = apple_stock['retweet_num'].values.reshape(-1,1)

scaler = MinMaxScaler()
closing = scaler.fit_transform(closing)
like = scaler.fit_transform(like)
comment = scaler.fit_transform(comment)
retweet = scaler.fit_transform(retweet)

In [68]:
from sklearn.model_selection import train_test_split
stack_np = np.column_stack((closing,like,comment,retweet))
X = pd.DataFrame(stack_np, columns=['closing','like','comment','retweet'])
y = apple_stock['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [69]:
model2 = Sequential()
model2.add(LSTM(units=50, return_sequences=True))
model2.add(LSTM(units=50))
model2.add(Dense(units=1))
model2.compile(loss='mse', optimizer='adam')
model2.compile(optimizer='adam', loss='mse',
              metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3778 - precision_9: 0.2529 - recall_9: 0.0481        
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2636 - precision_9: 0.5380 - recall_9: 0.7640 
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2509 - precision_9: 0.5990 - recall_9: 0.3558
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2504 - precision_9: 0.5286 - recall_9: 0.7505
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2501 - precision_9: 0.4990 - recall_9: 0.4974
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2472 - precision_9: 0.5409 - recall_9: 0.8667
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2520 - precision_9: 0.5386 - recall_9: 0.3346 
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2500 - precision_9: 0.5372 - recall_9: 0.9184 
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2502 - precision_9: 0.5295 - recall_9: 0.5147
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━

In [70]:
loss,precision,recall = model.evaluate(X_test, y_test)
f1 = 2 * (precision * recall) / (precision + recall)
print(precision, recall, f1)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2521 - precision_9: 0.5257 - recall_9: 0.5605  
0.5259740352630615 0.5625 0.543624166035286


In [70]:
# 模型大小差距 Bert。 Github里有参考
# 轻量Bert只处理推特数据，另一个分支处理股票数据（LSTM），再合并
# word2vec：把文本转换为向量 扔到LSTM里 常用
# 如今这些包比较成熟，背后的原理不再关键
# grid search 超参 各种资料 （learning rate）
# 可以放入探索过程